In [38]:
import tabula
from pathlib import Path
import pandas as pd
import re #modulo usado para buscar expressões específicas, basicamente uma busca eficaz de strings dentro de algo, no nosso caso, o PDF

pd.set_option('display.max_columns', None) # os 3 set_options abaixo são usados apenas para que as opções de retorno das informações das tabelas sejam completamente retiradas. Parâmetros como o máximo de colunas, linhas e o tamanho máximo das tabelas são diretamente retirados.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

pdf_path = Path("Modelo_PDF (1).pdf") 
tabelas = tabula.read_pdf(str(pdf_path), pages="1", multiple_tables=True, guess=False) # método usado para ler a tabela dentro do caminho da variavel "pdf_path" da pagina solicitada, dizendo que existem multiplas tabelas no mesmo pdf/campo


def extrair_valor(tabela, texto):
    for coluna in tabela.columns:
        if texto in coluna: #tentavida de encontrar o texto requerido na coluna
            valor = tabela[coluna].values[0]
            if isinstance(valor, str): #o isinstance é usado para verificar o tipo de dado que está sendo extraindo, no caso, usamos a variavel valor para ver se o seu tipo é igual a "str"
                match = re.search(r'(\d{2}/\d{2}/\d{4})', valor) # Usa o re para extrair apenas a data que tenha 2 num inteiros no primeiro "/", no segundo e 4 no segundo
                if match:
                    return match.group(1)#se a extração for bem-sucedida então o grupo, que é o valor do match, é retornado
            return valor
        else:
            # Se não encontrar na coluna, procura nas células
            for _, row in tabela.iterrows(): # o "_" é usado pra ignorar o indice, pq a informação que queremos pode estar em qualquer celula da tabela. E o iterrows é usado para retornar um iterador em pares
                for cell in row: #para percorrer cada celula dentro da coluna
                    if isinstance(cell, str) and texto in cell: #o isinstance é usado para verificar o tipo de dado que está sendo extraindo, no caso, usamos a variavel valor para ver se o seu tipo é igual a "str"
                        match = re.search(r'(\d{2}/\d{2}/\d{4})', cell) # Usa o re para extrair apenas a data que tenha 2 num inteiros no primeiro "/", no segundo e 4 no segundo
                        if match:
                            return match.group(1)#se a extração for bem-sucedida então o grupo, que é o valor do match, é retornado
    
    return None

if tabelas:
    for i, tabela in enumerate(tabelas):
        print(f"Tabela {i + 1}:")
        tabela = tabela.fillna('')  # para retirar todo espaço em branco que existe nas tabelas
        for coluna in tabela.columns:
            if 'Unnamed' in coluna:
                tabela['CURSO: Técnico em Desenvolvimento de Sistemas'] += ' ' + tabela[coluna].astype(str)#garante que todos os valores das tabelas presentes no arquivo pdf sejam mostrados corretamente. Além de concatenar as informações das outras colunas existentes que são chamadas de "Unnamed"
                tabela.drop(columns=[coluna], inplace=True) #após o retorno das tabelas, a sua visualização pode ser feita de maneira defeituosa, provocando desorganização, com o .drop, garantimos que toda coluna desnecessária seja removida.
        print(tabela)
        data_inicio = extrair_valor(tabela, "DATA INÍCIO")
        if data_inicio:
            print("A data de início é:", data_inicio)
        else:
            print("Data de início não encontrada")
        print("\n" + "-" * 50)
        
else:
    print("Nenhuma tabela encontrada no PDF")

Tabela 1:
                                          CURSO: Técnico em Desenvolvimento de Sistemas  \
0                    TURMA: G91118  DATA INÍCIO: 31/01/2024 DATA FIM: 17/06/2024  20241   
1                              DISCIPLINA: Lógica de Programação   CARGA HORÁRIA:  162    
2   PROFESSOR: ADS957 - Emerson Lins Guimarães   COORDENADOR:  Jailson José dos Santos    
3                                           REGISTRO DE AULAS - PORTAL DO DOCENTE         
4                                                               1 ABSTRAÇÃO LÓGICA        
5                                                             1.1 Álgebra Booleana        
6                          1.2 Fluxogramas, organogramas e representações gráficas        
7                                                                 2 TIPOS DE DADOS        
8                                                       2.1 Variáveis e constantes        
9                                               3 EXPRESSÕES LÓGICAS E ARITMÉTIC